In [1]:
!pip install uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 82.7 MB/s eta 0:00:00:00:0100:01


In [1]:
import torch

print(torch.cuda.is_available())

True


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() 
                      else "mps" if torch.backends.mps.is_available() 
                      else "cpu")

print(f"Current device: {device}")


Current device: cuda


In [5]:
import torch.nn as nn
import torch.optim as optim


In [15]:

class LSTM(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers,output_size,dropout= 0.2):
        super(LSTM,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size

        #Lets now add the lstm layer with bi directional as true
        self.lstm  = nn.LSTM(input_size,hidden_size,num_layers,dropout=dropout,batch_first=True,bidirectional=True)
        self.D = 2 if self.lstm.bidirectional else 1
        self.fc = nn.Linear(self.D*hidden_size,output_size)

    def forward(self,x):
        
        
        h0 = torch.zeros(self.D*self.num_layers,x.size(0),self.hidden_size).to(x.device)
        
        c0 = torch.zeros(self.D*self.num_layers,x.size(0),self.hidden_size).to(x.device)

        out ,(h,c) = self.lstm(x, (h0,c0))

        return self.fc(out[:,-1,:])
        



In [18]:
X = torch.randn(100,4,2)
Y = torch.randn(100,2)
criterion = nn.MSELoss()
model = LSTM(input_size=2,hidden_size=5,output_size=2,num_layers=2)
optimizer = optim.AdamW(model.parameters(),lr=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min')


for epoch in range(200):
    y_preds = model(X)
    loss = criterion(y_preds,Y)
    

    optimizer.zero_grad()
    loss.backward()
    nn.utils.clip_grad.clip_grad_norm_(model.parameters(),max_norm=0.1)
    optimizer.step()

    # loss calculation
    if epoch%5==0: print(f'Epoch {epoch}: Loss = {loss.detach().item()}')







Epoch 0: Loss = 1.0167508125305176
Epoch 5: Loss = 0.9537746906280518
Epoch 10: Loss = 0.939974308013916
Epoch 15: Loss = 0.9233512878417969
Epoch 20: Loss = 0.9080749750137329
Epoch 25: Loss = 0.898862898349762
Epoch 30: Loss = 0.885152280330658
Epoch 35: Loss = 0.8726259469985962
Epoch 40: Loss = 0.8785070180892944
Epoch 45: Loss = 0.869627833366394
Epoch 50: Loss = 0.8422443270683289
Epoch 55: Loss = 0.8226364850997925
Epoch 60: Loss = 0.8085905313491821
Epoch 65: Loss = 0.7739742398262024
Epoch 70: Loss = 0.7603943347930908
Epoch 75: Loss = 0.7374916076660156
Epoch 80: Loss = 0.7020481824874878
Epoch 85: Loss = 0.6897179484367371
Epoch 90: Loss = 0.6510323286056519
Epoch 95: Loss = 0.675072193145752
Epoch 100: Loss = 0.6594898104667664
Epoch 105: Loss = 0.6191093325614929
Epoch 110: Loss = 0.5654346346855164
Epoch 115: Loss = 0.5528398156166077
Epoch 120: Loss = 0.494335412979126
Epoch 125: Loss = 0.509710431098938
Epoch 130: Loss = 0.4871733784675598
Epoch 135: Loss = 0.4928238689